In [1]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, MaxPooling2D,UpSampling2D
from keras.utils import np_utils
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
#Seed for reproducibilty
np.random.seed(1338)

Using Theano backend.


In [2]:
%%time
#Loading the training and testing data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
img_rows, img_cols = 28, 28
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

CPU times: user 1.27 s, sys: 104 ms, total: 1.37 s
Wall time: 1.37 s


In [3]:
%%time
#Selecting 6000 random examples from the test data
test_rows = np.random.randint(0,X_test.shape[0],6000)
X_test = X_test[test_rows]

#Selecting the 5918 examples where the output is 6
X_six = X_train[y_train == 6]
#Selecting the examples where the output is not 6
X_not_six = X_train[y_train != 6]

#Selecting 6000 random examples from the data that contains only the data where the output is not 6
random_rows = np.random.randint(0,X_six.shape[0],6000)
X_not_six = X_not_six[random_rows]

CPU times: user 40 ms, sys: 32 ms, total: 72 ms
Wall time: 71.5 ms


In [4]:
%%time
#Appending the data with output as 6 and data with output as not six
X_train = np.append(X_six,X_not_six)
#Reshaping the appended data to appropraite form
X_train = X_train.reshape(X_six.shape[0] + X_not_six.shape[0], 1, img_rows, img_cols)

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 16.4 ms


In [5]:
%%time
#Input Dimensions
input_img = Input(shape=(1, 28, 28))
x = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(input_img)
encoded = MaxPooling2D((2, 2), border_mode='same')(x)

x = Convolution2D(8, 3, 3, activation='relu',border_mode='same')(encoded)
x = UpSampling2D((2, 2))(x)
decoded = Convolution2D(1, 3, 3, activation='sigmoid', border_mode='same')(x)

autoencoder = Model(input_img, decoded)

CPU times: user 400 ms, sys: 40 ms, total: 440 ms
Wall time: 502 ms


In [6]:
%%time
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 29.1 ms


In [7]:
%%time
autoencoder.fit(X_train, X_train,
                nb_epoch=15,
                batch_size=128,
                shuffle=True,
                validation_data=(X_test, X_test))

Train on 11918 samples, validate on 6000 samples
Epoch 1/15
11918/11918 [==============================] - 8s - loss: 0.1874 - val_loss: 0.0897
Epoch 2/15
11918/11918 [==============================] - 8s - loss: 0.0850 - val_loss: 0.0803
Epoch 3/15
11918/11918 [==============================] - 9s - loss: 0.0791 - val_loss: 0.0754
Epoch 4/15
11918/11918 [==============================] - 9s - loss: 0.0766 - val_loss: 0.0737
Epoch 5/15
11918/11918 [==============================] - 9s - loss: 0.0751 - val_loss: 0.0735
Epoch 6/15
11918/11918 [==============================] - 11s - loss: 0.0742 - val_loss: 0.0740
Epoch 7/15
11918/11918 [==============================] - 10s - loss: 0.0733 - val_loss: 0.0716
Epoch 8/15
11918/11918 [==============================] - 10s - loss: 0.0729 - val_loss: 0.0717
Epoch 9/15
11918/11918 [==============================] - 11s - loss: 0.0722 - val_loss: 0.0706
Epoch 10/15
11918/11918 [==============================] - 10s - loss: 0.0720 - val_loss: 0.

In [8]:
%%time
decoded_imgs = autoencoder.predict(X_test)

CPU times: user 2.86 s, sys: 32 ms, total: 2.9 s
Wall time: 1.53 s


In [9]:
%%time
# use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

CPU times: user 1.76 s, sys: 36 ms, total: 1.79 s
Wall time: 3.73 s
